# Étude de cas de location de courte durée Airbnb Bordeaux

![](airbnb.jpg)

##### Depuis l’an 2018 la ville de Bordeaux vit une pénurie de logements, de nombreux d’articles de la presse y sont consacrés  en pointant du doigt les locations courte durée qui sont largement dominé par le géant AirBnB, ce marché est réglementé par la municipalité mais est ce suffisant pour pour donner le souffle? Pour cette raison il est important de voir la situation actuel un peu plus en détails. Pour cela j’ai effectué une  récupération des données qui datent du 25 mai 2020 depuis le site insideairbnb.com


#### Importation des librairies

In [5]:
from IPython.core.display import display, HTML
import IPython.display
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import folium
from folium.plugins import FastMarkerCluster

from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import chart_studio.plotly as py
import plotly.graph_objects as go

quartiers = pd.read_csv("listings.csv", index_col= "id")

quartiers_details = pd.read_csv("listings_raw.csv", index_col= "id", low_memory=False)

calendrier = pd.read_csv("calendar.csv", parse_dates=['date'], index_col=['listing_id'])

reviews_details = pd.read_csv("reviews.csv", parse_dates=['date'])

ModuleNotFoundError: No module named 'plotly'

In [ ]:
print(quartiers.shape) # le fichier est composé de 10751 lignes

In [ ]:
target_columns = ["property_type", "accommodates", "first_review", "review_scores_value", "review_scores_cleanliness", "review_scores_location", "review_scores_accuracy", "review_scores_communication", "review_scores_checkin", "review_scores_rating", "maximum_nights", "listing_url", "host_is_superhost", "host_about", "host_response_time", "host_response_rate", "street", "weekly_price", "monthly_price", "market"]
quartiers = pd.merge(quartiers, quartiers_details[target_columns], on='id', how='left')
quartiers.info()


In [ ]:
quartiers.head(5) # Visualisation de donées que je vais traiter

### Visualisation des logements disponible par nombre et par quartier

In [ ]:
lieux=quartiers['neighbourhood'].value_counts().sort_values(ascending=True)
lieux.plot.barh(figsize=(15, 15), color='c', width=1)
plt.title("Le nombre de logements par quartier Bordeaux et agglomeration", fontsize=20)
plt.xlabel('La quantité', fontsize=14)
plt.show()

### Création de la carte interactive via folium

In [ ]:
lats2020 = quartiers['latitude'].tolist() # recuperation des donées des colonnes
long2020 = quartiers['longitude'].tolist()
locations = list(zip(lats2020, long2020))

map1 = folium.Map(location=[44.8323068, -0.65200609], zoom_start=12) # coordonées de la vue du départ
FastMarkerCluster(data=locations).add_to(map1) # nom du style de la carte
map1

### Quels types de logements sur Bordeaux et son aglomeration

#### Une remarque : à Bordeaux il est interdit de louer son logement pour une courte durée de plus de 120 jours.

In [ ]:
type_log = quartiers['room_type']. value_counts().sort_values(ascending=True)
type_log.plot.barh(figsize=(15, 3), width=1, color = ["#2E2D4D","#337357","#6D9F71", "#E4E3D3"]) # les couleurs personelles
plt.show()

#### Recuperation des noms des colonnes et leurs caractèrestiques 


In [ ]:
quartiers.nunique()

In [ ]:
quartiers['host_id'].nunique() # Quelques 8871 gérants pour 10751 biens 

### Voyons un peu plus en détails qu'est ce qui domine le marché de la location courte durée.

##### Ici je récupère le nombre de biens par le numéro identifiant. Tout de suite on remarque un identifiant qui saute aux yeux avec 105 biens.

In [ ]:
prop = pd.value_counts(quartiers.host_id).head(15)
prop

#### Voyons plus en details qui est derrière cet Id.

##### Dans ce tableau nous avons quelques informations sur les biens, notamment identifiant du bien, le titre de l'annonce et la position geographique.

In [ ]:
gourmand = quartiers[quartiers['host_id']== 160814130]
gourmand = gourmand[['name','host_id', 'host_name', 'latitude', 'longitude']]
gourmand.index.name = "listing_id"
gourmand.head()

In [ ]:
freq = quartiers.groupby(['host_id', 'host_name', 'host_about']).size().reset_index(name='num_host_quartiers')
freq = freq.sort_values(by=['num_host_quartiers'], ascending=False)
freq = freq[freq['num_host_quartiers'] >= 10] # Nombre de biens supérieur à ...
freq

#### Petit zoom sur les prix moyens à la nuitée par quartier

In [ ]:
feq = quartiers[quartiers['accommodates']==2]
feq = feq.groupby('neighbourhood')['price'].mean().sort_values(ascending=True)
feq.plot.barh(figsize=(10, 12), color='b', width=1)
plt.title("Le prix moyen à la nuitée pour 2 personnes", fontsize=20)
plt.xlabel('Prix à la nuitée (Euro)', fontsize=12)
plt.ylabel("")
plt.show()

### Hors sujet mais il est interessant de voir les notations des clients

In [ ]:
fig = plt.figure(figsize=(20,17))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=20)

ax1 = fig.add_subplot(121)
feq = quartiers[quartiers['number_of_reviews']>=10]
feq1 = feq.groupby('neighbourhood')['review_scores_location'].mean().sort_values(ascending=True)
ax1=feq1.plot.barh(color='b', width=1)
plt.title("La note moyenne sur 10", fontsize=15)
plt.xlabel('La note (1 à 10)', fontsize=20)
plt.ylabel("")

ax2 = fig.add_subplot(122)
feq = quartiers[quartiers['accommodates']==2]
feq2 = feq.groupby('neighbourhood')['price'].mean().sort_values(ascending=True)
ax2=feq2.plot.barh(color='b', width=1)
plt.title("Prix moyen à la nuitée pour 2 personnes", fontsize=20)
plt.xlabel('Prix moyen en (Euro)', fontsize=20)
plt.ylabel("")

plt.tight_layout()
plt.show()

### Les notes par categorie

In [3]:
listings10 = quartiers[quartiers['number_of_reviews']>=10]

fig = plt.figure(figsize=(20,15))
plt.rc('xtick', labelsize=16) 
plt.rc('ytick', labelsize=16)

ax1 = fig.add_subplot(321)
feq=listings10['review_scores_location'].value_counts().sort_index()
ax1=feq.plot.bar(color='b', width=1, rot=0)
#ax1.tick_params(axis = 'both', labelsize = 16)
plt.title("Emplacement", fontsize=24)
plt.ylabel('Nombre davis', fontsize=14)
plt.xlabel('La note moyenne', fontsize=14)

ax2 = fig.add_subplot(322)
feq=listings10['review_scores_cleanliness'].value_counts().sort_index()
ax2=feq.plot.bar(color='b', width=1, rot=0)
plt.title("Propreté", fontsize=24)
plt.ylabel('Nombre davis', fontsize=14)
plt.xlabel('La note moyenne', fontsize=14)

ax3 = fig.add_subplot(323)
feq=listings10['review_scores_value'].value_counts().sort_index()
ax3=feq.plot.bar(color='b', width=1, rot=0)
plt.title("Qualité-prix", fontsize=24)
plt.ylabel('Nombre davis', fontsize=14)
plt.xlabel('La note moyenne', fontsize=14)

ax4 = fig.add_subplot(324)
feq=listings10['review_scores_communication'].value_counts().sort_index()
ax4=feq.plot.bar(color='b', width=1, rot=0)
plt.title("Communication", fontsize=24)
plt.ylabel('Nombre davis', fontsize=14)
plt.xlabel('La note moyenne', fontsize=14)

ax5 = fig.add_subplot(325)
feq=listings10['review_scores_checkin'].value_counts().sort_index()
ax5=feq.plot.bar(color='b', width=1, rot=0)
plt.title("Arrivée", fontsize=24)
plt.ylabel('Nombre davis', fontsize=14)
plt.xlabel('La note moyenne', fontsize=14)

ax6 = fig.add_subplot(326)
feq=listings10['review_scores_accuracy'].value_counts().sort_index()
ax6=feq.plot.bar(color='b', width=1, rot=0)
plt.title("Précision", fontsize=24)
plt.ylabel('Nombre davis', fontsize=14)
plt.xlabel('La note moyenne', fontsize=14)

plt.tight_layout()
plt.show()

NameError: name 'quartiers' is not defined

In [4]:
quartiers.host_is_superhost = quartiers.host_is_superhost.replace({"t": "True", "f": "False"})

feq=quartiers['host_is_superhost'].value_counts()
feq.plot.bar(figsize=(10, 8), width=1, rot=0)
plt.title("Le nombre des status Super Host", fontsize=20)
plt.ylabel('Nombre de propriétères', fontsize=12)
plt.show()

NameError: name 'quartiers' is not defined